<a href="https://colab.research.google.com/github/pizzapuppies/Usif_FYP_code/blob/main/RD%20w/%20hypertuning%20%2B%20k-fold%20cross%20validation%20CNN_for_dengue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
! pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 34.4 MB/s 


In [104]:
from google.colab import drive
import pandas as pd 
import os
import tensorflow as tf
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import csv
import random
from tensorflow import keras
import sys
import keras_tuner as kt
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [105]:
mat = [[1,2,3],[4,5,6]]

def split23(li):
  g = []
  for i in range(1,len(li),3):
    g.append(li[i:i+3])
  return g

In [106]:
X = []

with open('/content/gdrive/My Drive/FYP Dengue data - Sheet1.csv', 'r') as f:
  csv_reader = csv.reader(f, delimiter=',')
  line_count = 0
    
  for i in csv_reader:
    if line_count == 0:
      for j in range(len(split23(i))):
        X.append([])
      
    line_count += 1
    if i[0] in ["index","conc","label"]:
            if i[0] == "label":
              y = split23(i)
    else:
      li = split23(i)
      
      for ind,dat in enumerate(li):
        com = list(map(float,dat[1:]))
        X[ind].append(complex(com[0],com[1]))
        



for i,li in enumerate(y):
  if li[0] == "1":
    y[i] = 1
  else:
    y[i] = 0

X = np.array(X)
y = np.array(y)

temp = list(zip(X, y))
random.shuffle(temp)
X, y = zip(*temp)
X, y = list(X), list(y)
X = np.array(X).astype('complex')
y = np.array(y)

In [107]:
def model_builder(hp):
  model = tf.keras.models.Sequential()
  model.add(keras.layers.Dense(128,         #Number of nodes
                        input_shape=(50,1), #Number of input variables
                        name='Hidden-Layer-1', #Logical name
                        activation='relu'))    #activation function

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  
  #Add a second hidden layer
  model.add(keras.layers.Dense(units=hp_units,
                                name='Hidden-Layer-2',
                                activation='relu'))

  model.add(tf.keras.layers.Conv1D(
      2,3,activation='relu'
  ))
  model.add(tf.keras.layers.Flatten())

  #Add a second hidden layer
  model.add(keras.layers.Dense(128,
                                name='Hidden-Layer-3',
                                activation='relu'))



  #Add a second hidden layer
  model.add(keras.layers.Dense(128,
                                name='Hidden-Layer-4',
                              activation='relu'))
  
  model.add(keras.layers.Dense(1,
                             name='Output-Layer',
                             activation='sigmoid'))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  
  

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=['accuracy'])

  return model

def learn_test(X_train,Y_train,X_test,Y_test):
  

  tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

  tuner.search(X_train, Y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

  # Get the optimal hyperparameters
  best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

  #print(f
  """
  The hyperparameter search is complete. The optimal number of units in the first densely-connected
  layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
  is {best_hps.get('learning_rate')}.
  )"""

 # Build the model with the optimal hyperparameters and train it on the data for 50 epochs
  model = tuner.hypermodel.build(best_hps)
  history = model.fit(X_train, Y_train, epochs=50, validation_split=0.2)

  val_acc_per_epoch = history.history['val_accuracy']
  best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
  print('Best epoch: %d' % (best_epoch,))

  hypermodel = tuner.hypermodel.build(best_hps)

  # Retrain the model
  hypermodel.fit(X_train, Y_train, epochs=best_epoch, validation_split=0.2)

  eval_result = hypermodel.evaluate(X_test, Y_test)
  print("[test loss, test accuracy]:", eval_result)
  return eval_result[1]



In [110]:
from sklearn.model_selection import KFold

n_split=10

li = []

for train_index,test_index in KFold(n_split).split(X):
  x_train,x_test=X[train_index],X[test_index]
  y_train,y_test=y[train_index],y[test_index]
  xa = learn_test(x_train,y_train,x_test,y_test)
  li.append(round(xa*100))

Epoch 1/50
4/4 [==============================] - 4s 300ms/step - loss: 0.6932 - accuracy: 0.4600 - val_loss: 0.6931 - val_accuracy: 0.3846
Epoch 2/50
4/4 [==============================] - 0s 72ms/step - loss: 0.6931 - accuracy: 0.5400 - val_loss: 0.6932 - val_accuracy: 0.3846
Epoch 3/50
4/4 [==============================] - 0s 91ms/step - loss: 0.6931 - accuracy: 0.5400 - val_loss: 0.6932 - val_accuracy: 0.3846
Epoch 4/50
4/4 [==============================] - 0s 78ms/step - loss: 0.6931 - accuracy: 0.5400 - val_loss: 0.6932 - val_accuracy: 0.3846
Epoch 5/50
4/4 [==============================] - 0s 74ms/step - loss: 0.6931 - accuracy: 0.5400 - val_loss: 0.6932 - val_accuracy: 0.3846
Epoch 6/50
4/4 [==============================] - 0s 79ms/step - loss: 0.6931 - accuracy: 0.5400 - val_loss: 0.6932 - val_accuracy: 0.3846
Epoch 7/50
4/4 [==============================] - 0s 63ms/step - loss: 0.6931 - accuracy: 0.5400 - val_loss: 0.6933 - val_accuracy: 0.3846
Epoch 8/50
4/4 [==========

In [109]:
print(max(li))

1.0


In [111]:
print(li)

[43, 29, 93, 100, 43, 36, 43, 29, 36, 50]
